In [1]:
import os.path
import re
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import requests

from pandas.io.json import json_normalize


%matplotlib inline

import os
import sys

try:
    from urllib.parse import urlparse
except ImportError:
     from urlparse import urlparse

import seaborn as sns
import numpy as np

DATA_DIR = '../data/data_processed/' 

In [2]:
articles = pd.read_csv(DATA_DIR + 'articles_popularity_score.csv', index_col=0)
articles.index.names = ['id']

In [3]:
ordered_articles = articles.sort_values(by="popularity_score").reset_index()

In [4]:
ordered_articles = ordered_articles.reset_index()
ordered_articles = ordered_articles.rename(columns={'index': 'ranking'})
ordered_articles

,ranking,id,title,categories,article_lenght,views,refs_count,link_count,death,locations,end_date_clean,ongoing,popularity_score
0,0,49532277,Hashimiya attack,civilian attack,0.000558,0.000021,0.000,0.000687,75,Hashimiya|Iraq,2014.0,False,0.001267
1,1,58421339,Fifth Third Center shooting,civilian attack,0.000431,0.000043,0.001,0.000687,0,NaN,0.0,False,0.002161
2,2,49429205,January 2015 Maiduguri suicide bombing,civilian attack,0.000470,0.000011,0.001,0.002062,19,Maiduguri|Nigeria,0.0,False,0.003543
3,3,50567259,Transkei War,military conflict,0.000786,0.000019,0.000,0.002749,7000,South Africa Q258,1836.0,False,0.003554
4,4,56640476,Operation of Sidi Aïch,military conflict,0.002870,0.000005,0.001,0.000000,0,Sidi Aïch|Tunisia,2015.0,False,0.003875
5,5,32166814,Conquest of Ai,military conflict,0.000483,0.000021,0.000,0.003436,0,Ai Q403166,1400.0,False,0.003940
6,6,51033965,Punjab War,military conflict,0.000404,0.000205,0.000,0.003436,0,Punjab,1820.0,False,0.004046
7,7,41345416,Battle of Deefa,military conflict,0.000566,0.000089,0.000,0.003436,0,Deefa|Oman,1968.0,False,0.004092
8,8,3216047,Sack of Dun Gallimhe,military conflict,0.000182,0.000037,0.000,0.004124,0,NaN,1247.0,False,0.004343
9,9,49532230,5 February 2014 Baghdad bombings,civilian attack,0.001437,0.000015,0.001,0.002062,32,Baghdad|Iraq,2014.0,False,0.004514


### recupero dati dai results

In [5]:
r = requests.get('http://ada.ineeda.space/getResults.php')
print('Response body (parsed json):')
jsonResults = r.json()
results_pd = json_normalize(jsonResults)

Response body (parsed json):


In [6]:
results_pd[['ref1','ref2']]

,ref1,ref2
0,34181799,4764461
1,34276,34166492
2,53559919,52019122
3,13846188,32496189
4,43227423,49069111
5,12336420,52048842
6,24821786,19345883
7,201936,21493140
8,46230181,3832376
9,32964156,771


In [7]:
results_pd['ref1'] = results_pd.ref1.astype(int)
results_pd['ref2'] = results_pd.ref2.astype(int)
results_pd['result'] = results_pd.result.astype(int)

duplicati

In [33]:
a = results_pd.groupby(['ref1','ref2']).count()
a[a.result > 1]

result
ref1     ref2            
0        48545523       2
771      252846         2
         40950218       2
863      52717005       2
32611    57470685       2
32927    30207234       2
70181    140376         2
201936   51035921       2
799679   36141091       2
1021687  863            2
1059855  4764461        2
1439685  46215853       2
1652975  14049933       2
1856025  47894039       2
3030955  25892827       2
3257081  36494808       2
3770751  43227423       3
4006553  7746616        2
5043324  401909         2
         4478023        2
5137464  41181297       2
5357959  36197          2
6299613  46973028       5
7409287  36197          4
7746616  277783         2
7761623  32927          2
10396793 6730717        2
10852367 200987         9
11455647 22623199       2
12359069 32927          2
12697212 20855562       2
13425800 28160913       2
13735436 863            2
18423367 5058690        2
18842471 50792014       2
20683060 32611          2
22937234 32927          2
24417962 6750583        2
25566083 46230181       2
26397119 5043324        2
33800638 32611          2
34472007 51046467       2
42563890 6750583        2
43517689 43060786       2
44855655 41181297       3
45546772 863            2
48581121 799679         2
48958544 8442323        3
51076275 3171634        2
52019122 56704492       2
52048588 47894039       2
53415419 464071         2
58112781 4764461       13

In [34]:
results_pd[results_pd[['ref1','ref2']].duplicated(keep=False)]

,ref1,ref2,result
118,33800638,32611,2
119,33800638,32611,2
228,44855655,41181297,0
229,44855655,41181297,0
230,44855655,41181297,0
242,58112781,4764461,2
243,58112781,4764461,2
244,58112781,4764461,2
245,58112781,4764461,2
246,58112781,4764461,0


In [61]:
duplicated_res = results_pd[(results_pd.result != 0) & (results_pd[['ref1','ref2']].duplicated()) & ~(results_pd[['ref1','ref2','result']].duplicated())]
duplicated_res

,ref1,ref2,result
336,32611,57470685,1
881,34472007,51046467,2
1454,3030955,25892827,1
1533,0,48545523,2


ci sono 4 che sono duplicati e che si contraddicono.. sono 8 su 2000 e passa, io li tolgo

In [62]:
for index, row in duplicated_res.iterrows():
    results_pd = results_pd.drop(results_pd[(results_pd.ref1 == row.ref1) & (results_pd.ref2 == row.ref2)].index)

verifico che non ci siano più quei duplicati

In [63]:
results_pd[(results_pd.result != 0) & (results_pd[['ref1','ref2']].duplicated()) & ~(results_pd[['ref1','ref2','result']].duplicated())]

,ref1,ref2,result


perfetto!

## Inizio il ranking

In [72]:
ordered_articles[ordered_articles['id'] == 23497381].ranking.item()

12761

In [79]:
totalOk = 0

for index, row in results_pd[results_pd['result'] != 0].iterrows():
    articleID1 = row['ref1']
    articleID2 = row['ref2']
    result = row['result']
    
    ranking1 = ordered_articles[ordered_articles['id'] == articleID1].ranking.item()
    ranking2 = ordered_articles[ordered_articles['id'] == articleID2].ranking.item()
    
    predictedResult = (1, 2)[ranking1 < ranking2]
    
    if(predictedResult == result):
        totalOk+=1

999

In [83]:
totalResponses = len(results_pd[results_pd['result'] != 0].index)

In [85]:
percCorrects = totalOk/totalResponses*100

In [86]:
percCorrects

94.3342776203966